In [ ]:

from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [ ]:
import fastbook
import torch
from fastai.vision.all import *
from fastbook import *
import numpy as np
import numpy.random as nprnd



fastbook.setup_book()


In [ ]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])
txt = files[0].open().read(); txt[:75]

In [ ]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))
tkn = Tokenizer(spacy)

In [ ]:
if tkn:
    coll_repr(tkn('©   Fast.ai www.fast.ai/INDEX'), 31)
    txts = L(o.open().read() for o in files[:2000])
     
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    sub = first(sp([txt]))
    if sub:
        return ' '.join(sub[:40])

subword(1000)


In [ ]:
if tkn:
    toks = tkn(txt)

    toks200 = txts[:200].map(tkn)
    toks200[0]
    num = Numericalize()
    num.setup(toks200)
    print(coll_repr(num.vocab,20))
    nums = num(toks)[:20]; 
    print(nums)
    print(' '.join(num.vocab[o] for o in nums))
    
     

In [ ]:
from IPython.display import HTML
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1"
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

In [ ]:
nums200 = toks200.map(num)

dl = LMDataLoader(nums200)

In [ ]:

x,y = first(dl)
x.shape,y.shape

' '.join(num.vocab[o] for o in x[0][:20])

In [ ]:

get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)
     

In [ ]:

dls_lm.show_batch(max_n=2)
     


In [ ]:
torch.device('cuda',0)
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
learn.fit_one_cycle(1, 2e-2)
     

In [ ]:
from shutil import copy
import os

# Get the current directory
current_dir = os.getcwd()

learn.save(f'{current_dir}/1epoch')


In [ ]:
current_dir = os.getcwd()

#learn.unfreeze()
#learn.fit_one_cycle(10, 2e-3)
#learn.save('1epoch')
learn.load(f'{current_dir}/1epoch')
     

In [ ]:
learn.save_encoder(f'{current_dir}/finetuned')

In [ ]:
TEXT = "I liked this movie because,the actor xxx Tauru "
N_WORDS = 40
N_SENTENCES = 1
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

print("\n".join(preds))

In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab), CategoryBlock),
    get_y=parent_label, get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)
dls_clas.vocab[1]

In [ ]:
dls_clas.show_batch(max_n=3)

In [ ]:
nums_sample = toks200[:10].map(num)

In [ ]:
nums_sample.map(len)

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,metrics=accuracy).to_fp16()

In [ ]:
learn = learn.load_encoder(f'{current_dir}/finetuned')

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
torch.device('cuda',0)
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

In [ ]:
# Make predictions on new text data
text = "that 01_intro.ipynb is a great movie!"
pred_class, pred_idx, outputs = learn.predict(text)

# Print the predicted class and outputs
print(f"Predicted class: {pred_class}; Outputs: {outputs}")

In [ ]:
learn.save(f'{current_dir}/classifier')